In [5]:
#1 - LIBRARIES

import pandas as pd
import numpy as np

from imblearn.over_sampling import SMOTE, RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler


from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler, QuantileTransformer, PowerTransformer, Normalizer
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import (
    confusion_matrix, precision_score, recall_score, accuracy_score,
    roc_auc_score, f1_score, log_loss, brier_score_loss, average_precision_score,  
    balanced_accuracy_score, matthews_corrcoef, classification_report
)

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.metrics import AUC, Precision, Recall, TruePositives, TrueNegatives, FalsePositives, FalseNegatives
from tensorflow.keras.callbacks import CSVLogger, EarlyStopping, ReduceLROnPlateau, ModelCheckpoint


from sklearn.metrics import (
    log_loss,                # substitui BinaryCrossentropy do TF
    roc_auc_score,
    confusion_matrix,
    precision_score,
    recall_score,
    accuracy_score,
    f1_score,
    balanced_accuracy_score,
    matthews_corrcoef
)
import numpy as np
import pandas as pd



import torch
from xgboost import XGBClassifier

from catboost import CatBoostClassifier, Pool

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

from sklearn.ensemble import RandomForestClassifier

from sklearn.compose import ColumnTransformer


In [ ]:
# 2 - LOADING DATA
separator = ','
fold_name = "4" # set the fold number for the files

url_train = "data/6h/train_df_kfold_"+fold_name+".csv"
url_val = "data/6h/val_df_kfold_"+fold_name+".csv"
url_test = "data/6h/test_df_kfold_"+fold_name+".csv"


# CHOOSE COLUMNS TO DELETE

qtde_attributes = 18 #18 10

#18 attributes
list_col_delete = ['Class_Flare','Letra_Class', 'T_REC', 'harpnum']

#10 Bobra'attributes
list_col_delete = ['Class_Flare','Letra_Class','T_REC', 'harpnum', 'MEANGAM','MEANGBH','MEANGBT','MEANGBZ', 'MEANJZD','MEANJZH','MEANALP','MEANSHR']

#10 SHAP'atrributes
list_col_delete = ['Class_Flare','Letra_Class', 'T_REC', 'harpnum' ,'ABSNJZH','MEANGAM','MEANJZD','MEANJZH','SAVNCPP','TOTPOT','TOTUSJH','TOTUSJZ']




# NORMALIZATION
scaler_name = 'StandardScaler'  #'StandardScaler', 'RobustScaler', 'MinMaxScaler', 'NormalizerL1', 'PowerTransformer', 'QuantileTransformer'


# BALANCING
balanceamento = 'smote'  #'smote', 'oversampling', 'undersampling', 'class_weight', ou 'none'

# MODEL 
model_name  = 'random_forest' #xgboost, catboost, random_forest, logistic_regression


epoch = 100
batch = 64

optimizer=keras.optimizers.Adam(learning_rate=1e-4) #1e4
focal_gamma = 2
focal_alpha = 0.25



In [14]:
# 3 - READ AND PREPARE DATA

train_df = pd.read_csv(url_train, sep=separator)
val_df = pd.read_csv(url_val, sep=separator)
test_df = pd.read_csv(url_test, sep=separator)


# Convert datetime
date1_ta = pd.to_datetime(train_df['T_REC'], errors='coerce', format='%Y-%m-%d %H:%M:%S')
date2_ta = pd.to_datetime(train_df['T_REC'], errors='coerce', format='%Y-%m-%d')

date1_va = pd.to_datetime(val_df['T_REC'], errors='coerce', format='%Y-%m-%d %H:%M:%S')
date2_va = pd.to_datetime(val_df['T_REC'], errors='coerce', format='%Y-%m-%d')

date1_te = pd.to_datetime(test_df['T_REC'], errors='coerce', format='%Y-%m-%d %H:%M:%S')
date2_te = pd.to_datetime(test_df['T_REC'], errors='coerce', format='%Y-%m-%d')

train_df['T_REC'] = date1_ta.fillna(date2_ta)
val_df['T_REC'] = date1_va.fillna(date2_va)
test_df['T_REC'] = date1_te.fillna(date2_te)

# Remove timezone to avoid date shifts
train_df['T_REC'] = train_df['T_REC'].dt.tz_localize(None)
val_df['T_REC'] = val_df['T_REC'].dt.tz_localize(None)
test_df['T_REC'] = test_df['T_REC'].dt.tz_localize(None)

# order date
train_df = train_df.sort_values(by='T_REC')
val_df = val_df.sort_values(by='T_REC')
test_df = test_df.sort_values(by='T_REC')


#save test extra columns
harpnum_test = test_df['harpnum'].values
t_rec_test = test_df['T_REC'].values
letra_class_test = test_df['Letra_Class'].values

info_test = pd.DataFrame({
    'harpnum': harpnum_test,
    'T_REC': t_rec_test,
    'Letra_Class': letra_class_test
})
info_test.to_csv('results/'+model_name+'-info_test-fold'+fold_name+'.csv', index=False)


#delete columns
for lcd in list_col_delete:
    train_df.pop(lcd)
    val_df.pop(lcd)
    test_df.pop(lcd)
    



C:\Users\Juliana Sabino\AppData\Local\Temp\ipykernel_58516\558553179.py:3: DtypeWarning: Columns (3,16) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv(url_train, sep=separator)
C:\Users\Juliana Sabino\AppData\Local\Temp\ipykernel_58516\558553179.py:4: DtypeWarning: Columns (3,16) have mixed types. Specify dtype option on import or set low_memory=False.
  val_df = pd.read_csv(url_val, sep=separator)
C:\Users\Juliana Sabino\AppData\Local\Temp\ipykernel_58516\558553179.py:5: DtypeWarning: Columns (3,16) have mixed types. Specify dtype option on import or set low_memory=False.
  test_df = pd.read_csv(url_test, sep=separator)


In [ ]:
# 4 - Prepare Data

# 1. Separar X e y
X_train = train_df.drop(columns='Class')
y_train = train_df['Class']

X_val = val_df.drop(columns='Class')
y_val = val_df['Class']

X_test = test_df.drop(columns='Class')
y_test = test_df['Class']

'''
letra_class_col = X_test['Letra_Class'].copy()
harpnum_col =  X_test['harpnum'].copy()
T_REC_col =  X_test['T_REC'].copy()

X_test_model = X_test.drop(columns=['Letra_Class', 'harpnum', 'T_REC'])
X_train_model = X_train.drop(columns=['Letra_Class', 'harpnum', 'T_REC'])
X_val_model = X_val.drop(columns=['Letra_Class', 'harpnum', 'T_REC'])
'''

X_test_model = X_test
X_train_model = X_train
X_val_model = X_val



# 5 - Normalization

scaler_name = "StandardScaler"
scaler_standard = StandardScaler()
scaler_robust = RobustScaler()
scaler_minmax = MinMaxScaler()
scaler_l1 = Normalizer(norm='l1')
transformer_yeo = PowerTransformer(method='yeo-johnson')
transformer_quantile = QuantileTransformer(output_distribution='normal')

# Choose the scaler based on the scaler_name variable
if scaler_name == 'StandardScaler':
    scaler = scaler_standard
elif scaler_name == 'RobustScaler':
    scaler = scaler_robust
elif scaler_name == 'MinMaxScaler':
    scaler = scaler_minmax
elif scaler_name == 'NormalizerL1':
    scaler = scaler_l1
elif scaler_name == 'PowerTransformer':
    scaler = transformer_yeo
elif scaler_name == 'QuantileTransformer':
    scaler = transformer_quantile
else:
    raise ValueError(f"Scaler '{scaler_name}' not recognized. Please choose a valid one.")




X_train_scaled = scaler.fit_transform(X_train_model)
X_test_scaled = scaler.transform(X_test_model)


# 6 -  Balancing

if balanceamento == 'smote':
    smote = SMOTE(sampling_strategy=0.6, k_neighbors=3, random_state=42)
    X_train_res, y_train_res = smote.fit_resample(X_train, y_train)
    class_weights = None  # not necessary in this case

elif balanceamento == 'oversampling':
    oversample = RandomOverSampler(sampling_strategy=0.6, random_state=42)
    X_train_res, y_train_res = oversample.fit_resample(X_train, y_train)
    class_weights = None

elif balanceamento == 'undersampling':
    undersample = RandomUnderSampler(sampling_strategy=0.6, random_state=42)
    X_train_res, y_train_res = undersample.fit_resample(X_train, y_train)
    class_weights = None

elif balanceamento == 'class_weight':
    # Does not change X_train/y_train, just calculates the weights
    X_train_res, y_train_res = X_train, y_train
    classes = np.unique(y_train)
    weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_train)
    class_weights = dict(zip(classes, weights))

elif balanceamento == 'none':
    X_train_res, y_train_res = X_train, y_train
    class_weights = None

else:
    raise ValueError("Invalid balancing method. Choose from: 'smote', 'oversample', 'undersample', 'class_weight', 'none'.")

# 7 - change types
for col in ['USFLUX', 'TOTPOT']:
    X_train_res[col] = pd.to_numeric(X_train_res[col], errors='coerce')
    X_val[col] = pd.to_numeric(X_train_res[col], errors='coerce')
    X_test[col] = pd.to_numeric(X_train_res[col], errors='coerce')
    

from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='median')
X_train_res[['USFLUX', 'TOTPOT']] = imputer.fit_transform(X_train_res[['USFLUX', 'TOTPOT']])
X_val[['USFLUX', 'TOTPOT']] = imputer.fit_transform(X_val[['USFLUX', 'TOTPOT']])
X_test[['USFLUX', 'TOTPOT']] = imputer.fit_transform(X_test[['USFLUX', 'TOTPOT']])



print(X_train_res.dtypes.loc[['USFLUX', 'TOTPOT']])
print(X_val.dtypes.loc[['USFLUX', 'TOTPOT']])
print(X_test.dtypes.loc[['USFLUX', 'TOTPOT']])

USFLUX    float64
TOTPOT    float64
dtype: object
USFLUX    float64
TOTPOT    float64
dtype: object
USFLUX    float64
TOTPOT    float64
dtype: object


In [18]:
# Models
if model_name == 'xgboost':

    # 8 - XGBoost
    neg = (y_train == 0).sum()
    pos = (y_train == 1).sum()
    scale_pos_weight = neg / pos



    model = XGBClassifier(
        # — núcleo do boosting —
        n_estimators       = 50,     # muitas árvores + learning_rate baixo
        learning_rate     = 0.02, 
        max_depth         = 8,
        min_child_weight  = 1,         # controla complexidade dos nós
        subsample         = 0.8,       # bagging interno
        colsample_bytree  = 0.8,       # aleatoriedade em colunas
        gamma             = 0.10,      # exige ganho mínimo p/ dividir nó

        # — regularização —
        reg_lambda        = 1.0,       # L2
        reg_alpha         = 0.10,      # L1

        # — dados desbalanceados —
        scale_pos_weight  = scale_pos_weight,  # ≈ 190-200 no seu conjunto

        # — miscelânea —
        objective         = "binary:logistic",
        eval_metric       = ["auc", "aucpr", "logloss"],   # AUC-PR é mais informativa que logloss aqui
        tree_method       = "hist",    # rápido em CPU
        random_state      = 42,
        n_jobs            = -1,
        use_label_encoder = False
    )

    # ----------------------------
    # 4) treino com early stopping
    # ----------------------------
    model.fit(
        X_train_res, y_train_res,
        eval_set=[(X_val, y_val)],
        #eval_metric="aucpr",
        #early_stopping_rounds=50,
        verbose=100          # imprime a cada 100 iterações
    )
elif model_name == "catboost":
    # 8 - Catboost
    neg = (y_train == 0).sum()
    pos = (y_train == 1).sum()
    scale_pos_weight = neg / pos


    cat_features = [] #because all fields are numeric

    train_pool = Pool(
        data=X_train_res,
        label=y_train_res,
        cat_features=cat_features
    )

    val_pool = Pool(
        data=X_val,
        label=y_val,
        cat_features=cat_features
    )


    model = CatBoostClassifier(
        # — núcleo do boosting —
        iterations        = 150,   #150       # "n_estimators" em XGBoost
        learning_rate     = 0.09, #0.02
        depth             = 7,            # "max_depth" 8
        min_data_in_leaf  = 2,            # "min_child_weight" ~, usa min_data_in_leaf
        subsample         = 0.8,          # idem
        rsm               = 0.8,          # "colsample_bytree"
        random_strength   = 0.10,         # ~ gamma (controle de splits aleatórios) 0.1

        # — regularização —
        l2_leaf_reg       = 1.0,          # reg_lambda
        bagging_temperature = 0.1,        # ajuda contra overfitting (semelhante a reg_alpha)

        # — dados desbalanceados —
        class_weights     = None,  # ou None se preferir sem

        # — miscelânea —
        loss_function     = 'Logloss',      # otimizado internamente; métricas separadas
        eval_metric       = "PRAUC",                 # **uma só**
        custom_metric     = ["AUC", "Logloss"],      # recebe “AUC”, “AUC:PR”, “Logloss” etc.
        random_seed       = 42,
        verbose           = 100,            # imprime a cada 100 iterações
        task_type         = 'CPU',          # mude para 'GPU' se tiver suporte
        # devices='0'     # se usar GPU, especifique o índice da sua GTX 1650
    )

    # -------------------------------------------------
    # 5) treino com early stopping (CatBoost = 'od' / overfitting detector)
    # -------------------------------------------------
    model.fit(
        train_pool,
        eval_set=val_pool,
        early_stopping_rounds=50,      # igual ao XGBoost
        use_best_model=True
    )
elif model_name == "random_forest":
    
    numeric_features = X_train.columns  # todos numéricos no seu caso

    preprocess = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(with_mean=False), numeric_features),  # evita zeros após SMOTE
        ],
        remainder='passthrough',
        verbose_feature_names_out=False
    )
    
    rf_clf = RandomForestClassifier(
    n_estimators=600,            # número de árvores
    max_depth=50,             # deixe crescer, mas…
    min_samples_split=2,
    min_samples_leaf=1,
    max_features='sqrt',        # raiz do nº de features por split
    class_weight='balanced',    # compensa a minoria (~5 %)
    n_jobs=-1,                  # usa todos os núcleos
    random_state=42,
    oob_score=True              # validação out‑of‑bag (opcional)
)

    model = Pipeline(steps=[
        ('prep', preprocess),
        ('clf', rf_clf)
    ])

    # Treino
    model.fit(X_train_res, y_train_res)
        
elif model_name == "logistic_regression":

    model = Pipeline([
    ("scaler", StandardScaler()),             # padroniza média 0, desvio‑padrão 1
    ("clf", LogisticRegression(
        penalty="l2",                         # regularização padrão
        C=1.0,                                # força da regularização (pode ajustar)
        solver="lbfgs",                       # bom para datasets médios/grandes
        max_iter=1000,                        # mais iterações p/ garantir convergência
        class_weight="balanced",              # compensa a minoria
        n_jobs=-1,                            # usa todos os núcleos
        random_state=42
    ))
])


    model.fit(X_train_res, y_train_res)




In [ ]:
# 9 - Model Evaluate

# ── (2) test evaluate ─────────────────────────────────
y_pred_probs = model.predict_proba(X_test)[:, 1]      
test_loss    = log_loss(y_test, y_pred_probs)        
test_auc     = roc_auc_score(y_test, y_pred_probs)

print(f"Test Results - LogLoss: {test_loss:.4f}, AUC: {test_auc:.4f}")

# ── (3) informações auxiliares do fold (inalterado) ───────────────────────────
info_test = pd.read_csv(f"results/"+model_name+"-info_test-fold"+fold_name+".csv")[
    ['harpnum', 'T_REC', 'Letra_Class']
].reset_index(drop=True)

metrics_list = []
positivos_info_total = []
probs_com_classe_real = []

# ── (4) CSV-3 – real class + %  ───────────────────────────────────
for idx, (true_class, prob) in enumerate(zip(y_test, y_pred_probs)):
    harpnum      = info_test.loc[idx, 'harpnum']
    t_rec        = info_test.loc[idx, 'T_REC']
    letra_class  = info_test.loc[idx, 'Letra_Class']
    probs_com_classe_real.append({
        'harpnum': harpnum,
        'classe_real': int(true_class),
        'probabilidade_modelo': round(float(prob), 6),
        'T_REC': t_rec,
        'Letra_Class': letra_class
    })

# ── (5)  loop % ────
thresholds = np.arange(0.10, 1.00, 0.01)

for threshold in thresholds:
    y_pred = (y_pred_probs >= threshold).astype(int)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

    # metrics
    computed_loss  = log_loss(y_test, y_pred_probs)        # ← agora com sklearn
    precision      = precision_score(y_test, y_pred, zero_division=0)
    recall         = recall_score(y_test, y_pred, zero_division=0)
    accuracy       = accuracy_score(y_test, y_pred)
    auc_score      = roc_auc_score(y_test, y_pred_probs)
    f1             = f1_score(y_test, y_pred, zero_division=0)
    balanced_acc   = balanced_accuracy_score(y_test, y_pred)
    mcc            = matthews_corrcoef(y_test, y_pred)

    # metrics
    sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    tss = sensitivity + specificity - 1
    total = tp + tn + fp + fn
    pe = ((tp + fn)*(tp + fp) + (tn + fn)*(tn + fp)) / (total**2)
    hss = (accuracy - pe) / (1 - pe) if (1 - pe) != 0 else 0
    fpr = fp / (fp + tn) if (fp + tn) > 0 else 0
    fnr = fn / (fn + tp) if (fn + tp) > 0 else 0

    if round(threshold, 2) == 0.5:
        print("threshold:", round(threshold, 2))
        print("TSS:", tss)
        print("HSS:", hss)
        print("TP:", tp, " TN:", tn, " FP:", fp, " FN:", fn)

    metrics_list.append({
        'threshold': round(threshold, 2),
        'tn': tn, 'fp': fp, 'fn': fn, 'tp': tp,
        'loss': computed_loss,
        'auc': auc_score,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'accuracy': accuracy,
        'balanced_accuracy': balanced_acc,
        'mcc': mcc,
        'tss': tss,
        'hss': hss,
        'true_positive_rate': sensitivity,
        'true_negative_rate': specificity,
        'false_positive_rate': fpr,
        'false_negative_rate': fnr
    })

    # CSV-2 – harpnums 
    for idx in np.where(y_pred == 1)[0]:
        harpnum = info_test.loc[idx, 'harpnum']
        letra_class = info_test.loc[idx, 'Letra_Class']
        positivos_info_total.append({
            'threshold': round(threshold, 2),
            'harpnum': harpnum,
            'probabilidade': round(float(y_pred_probs[idx]), 6),
            'Letra_Class': letra_class
        })

# ── (6) 
pd.DataFrame(metrics_list).to_csv(f"results/"+model_name+"-metrics-fold"+fold_name+".csv", index=False)

df_positivos = pd.DataFrame(positivos_info_total).drop_duplicates()
df_positivos.to_csv(f"results/"+model_name+"-harpnums_all_thresholds_fold"+fold_name+".csv", index=False)

pd.DataFrame(probs_com_classe_real).to_csv(
    f"results/"+model_name+"-real_class_prob_harpnum_"+fold_name+".csv", index=False
)

print("\nResults saved successfully.")


Test Results - LogLoss: 0.0516, AUC: 0.9298
threshold: 0.5
TSS: 0.04007045007316212
HSS: 0.0755182751585384
TP: 45  TN: 138718  FP: 15  FN: 1075

Resultados salvos com sucesso.
